In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
# Importamos o SMOTE da biblioteca imbalanced-learn
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import numpy as np # Para contar as classes

# ==============================================================================
# --- 1. Carregando e Pré-processando os Dados ---
# ==============================================================================
print("--- 1. Carregando e Pré-processando os Dados ---")

# ... (Seu código de carregamento e pré-processamento) ...
# (Ocultado por brevidade, mas é o mesmo código que você já tem)
# ... (Seu código de carregamento e pré-processamento) ...

# Carregar o dataset
file_path = '../data/raw/leish_dataset.csv'
df = pd.read_csv(file_path)

# Criar cópia para processamento
df_processed = df.copy()

# Lidar com valores ausentes (Missing)
for col in df_processed.select_dtypes(include=['object']).columns:
    df_processed[col] = df_processed[col].fillna('Unknown')

# Codificar a variável Alvo (Target)
target_map = {'positivo': 1, 'negativo': 0, 'Unknown': 0}
df_processed['diagnosis'] = df_processed['diagnosis'].map(target_map).astype(int)

# Separar features (X) e alvo (y)
X_categorical = df_processed.drop('diagnosis', axis=1)
y = df_processed['diagnosis']

# Aplicar One-Hot Encoding nas features categóricas
X_numeric = pd.get_dummies(X_categorical, drop_first=True, dtype=int)
feature_names = X_numeric.columns.tolist() # Salvar nomes das colunas

print("--- Pré-processamento Concluído ---")


# ==============================================================================
# --- 2. Dividindo os Dados (Treino e Teste) ---
# ==============================================================================
print("\n--- 2. Dividindo os Dados (Treino e Teste) ---")

X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, 
    y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Tamanho do conjunto de Treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de Teste:  {X_test.shape[0]}")


# ==============================================================================
# --- 3. Modelo 4: Aplicando SMOTE (Balanceamento dos Dados) ---
# ==============================================================================
print("\n--- 3. Aplicando SMOTE para balancear os dados de TREINO ---")

# Contar as classes ANTES do SMOTE
unique_before, counts_before = np.unique(y_train, return_counts=True)
print(f"Distribuição de classes ANTES do SMOTE: {dict(zip(unique_before, counts_before))}")

# Inicializar o SMOTE
# k_neighbors=5 é um padrão, mas como você tem poucos positivos (95), 
# se der erro, tente diminuir para k_neighbors=3 ou k_neighbors=1
try:
    sm = SMOTE(random_state=42, k_neighbors=5)
except ValueError:
    print("Aviso: Poucas amostras na classe minoritária. Tentando k_neighbors=3.")
    sm = SMOTE(random_state=42, k_neighbors=3)


# Aplicar SMOTE. IMPORTANTE: Aplicar APENAS nos dados de TREINO.
# Nunca aplique SMOTE nos dados de teste!
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

# Contar as classes DEPOIS do SMOTE
unique_after, counts_after = np.unique(y_train_smote, return_counts=True)
print(f"Distribuição de classes DEPOIS do SMOTE: {dict(zip(unique_after, counts_after))}")
print(f"Novo tamanho do conjunto de Treino (SMOTE): {X_train_smote.shape[0]}")


# ==============================================================================
# --- 4. Treinando o Modelo (Árvore) com Dados Balanceados (SMOTE) ---
# ==============================================================================
print("\n--- 4. Treinando Árvore de Decisão com dados do SMOTE ---")

# Inicializar o classificador (o nosso "campeão" anterior)
# AGORA NÃO PRECISAMOS de class_weight='balanced', pois os dados JÁ ESTÃO balanceados.
dt_smote = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,       # Mantendo a profundidade que funcionou bem
    random_state=42
)

# Treinar o modelo nos dados sintéticos
dt_smote.fit(X_train_smote, y_train_smote)

print("--- Treinamento do Modelo 4 (SMOTE) Concluído ---")


# ==============================================================================
# --- 5. Avaliando o Modelo 4 (SMOTE) ---
# ==============================================================================
print("\n--- 5. Avaliando o Modelo 4 (SMOTE) no conjunto de TESTE original ---")

# Fazer previsões no conjunto de TESTE (o original, sem SMOTE)
y_pred_smote = dt_smote.predict(X_test)

# Calcular a Acurácia
print(f"Acurácia (Modelo 4 - SMOTE): {accuracy_score(y_test, y_pred_smote):.4f}")

# Exibir Relatório de Classificação
print("\nRelatório de Classificação (Modelo 4 - SMOTE):")
print(classification_report(y_test, y_pred_smote, target_names=['negativo (0)', 'positivo (1)']))

# Exibir Matriz de Confusão
print("\nMatriz de Confusão (Modelo 4 - SMOTE):")
cm_smote = confusion_matrix(y_test, y_pred_smote)
print(f"            [Prev. Neg] [Prev. Pos]")
print(f"[Real Neg]  {cm_smote[0][0]:>10} {cm_smote[0][1]:>10}")
print(f"[Real Pos]  {cm_smote[1][0]:>10} {cm_smote[1][1]:>10}")

--- 1. Carregando e Pré-processando os Dados ---
--- Pré-processamento Concluído ---

--- 2. Dividindo os Dados (Treino e Teste) ---
Tamanho do conjunto de Treino: 319
Tamanho do conjunto de Teste:  137

--- 3. Aplicando SMOTE para balancear os dados de TREINO ---
Distribuição de classes ANTES do SMOTE: {np.int64(0): np.int64(224), np.int64(1): np.int64(95)}
Distribuição de classes DEPOIS do SMOTE: {np.int64(0): np.int64(224), np.int64(1): np.int64(224)}
Novo tamanho do conjunto de Treino (SMOTE): 448

--- 4. Treinando Árvore de Decisão com dados do SMOTE ---
--- Treinamento do Modelo 4 (SMOTE) Concluído ---

--- 5. Avaliando o Modelo 4 (SMOTE) no conjunto de TESTE original ---
Acurácia (Modelo 4 - SMOTE): 0.4599

Relatório de Classificação (Modelo 4 - SMOTE):
              precision    recall  f1-score   support

negativo (0)       0.68      0.43      0.53        96
positivo (1)       0.29      0.54      0.37        41

    accuracy                           0.46       137
   macro av